# Training Deep Learning

## Reference and Memo about this code 
- tensorflow version 
- please run with Colab pro+ environment
- https://towardsdatascience.com/tutorial-using-deep-learning-and-cnns-to-make-a-hand-gesture-recognition-model-371770b63a51
- https://github.com/filipefborba/HandRecognition/blob/master/project3/project3.ipynb

## Import 

In [ ]:
# Here we import everything we need for the project

%matplotlib inline
from google.colab import files
import os

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from tqdm import tqdm

# Sklearn
from sklearn.model_selection import train_test_split # Helps with organizing data for training
from sklearn.metrics import confusion_matrix # Helps present results as a confusion-matrix

# Import of keras model and hidden layers for convolutional network
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten

print(tf.__version__)

2.9.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = '/content/drive/MyDrive/Dolphin/Dataset/'

## Dataset Setting

- preprocessing 
- spliting 

In [ ]:
# original data -> data => dataset
data_01 = pd.read_csv(path+"Dataset01/gesture01.csv")
data_02 = pd.read_csv(path+"Dataset02/gesture02.csv")
data_03 = pd.read_csv(path+"Dataset03/gesture03.csv")
data_04 = pd.read_csv(path+"Dataset04/gesture04.csv")
data_05 = pd.read_csv(path+"Dataset05/gesture05.csv")
data_06 = pd.read_csv(path+"Dataset06/gesture06.csv")
data_07 = pd.read_csv(path+"Dataset07/gesture07.csv")
data_08 = pd.read_csv(path+"Dataset08/gesture08.csv")
dataset = [data_01, data_02, data_03, data_04,
           data_05, data_06, data_07, data_08]

data_08

,Unnamed: 0,0,1,2,3,4,0.1,1.1,2.1
0,0,HandLandmark.WRIST,Right,0.510125,0.346040,5.360000e-07,0.000000,0.000000,0.000000
1,1,HandLandmark.THUMB_CMC,Right,0.420649,0.381017,-2.213014e-02,-0.089476,0.034977,-0.022131
2,2,HandLandmark.THUMB_MCP,Right,0.356711,0.467652,-4.382345e-02,-0.153414,0.121612,-0.043824
3,3,HandLandmark.THUMB_IP,Right,0.321645,0.550401,-6.050245e-02,-0.188480,0.204361,-0.060503
4,4,HandLandmark.THUMB_TIP,Right,0.271087,0.589258,-8.189777e-02,-0.239039,0.243218,-0.081898
...,...,...,...,...,...,...,...,...,...
6295,6295,HandLandmark.RING_FINGER_TIP,Left,0.152015,0.867936,-1.295522e-01,-0.002602,0.348590,-0.129553
6296,6296,HandLandmark.PINKY_MCP,Left,0.116323,0.586861,-9.549548e-02,-0.038294,0.067515,-0.095497
6297,6297,HandLandmark.PINKY_PIP,Left,0.112135,0.695031,-1.175685e-01,-0.042482,0.175686,-0.117570
6298,6298,HandLandmark.PINKY_DIP,Left,0.115710,0.766805,-1.209618e-01,-0.038907,0.247459,-0.120963


In [ ]:
def preprocessed_data(data, y) :    
    # concating y value at the end of columns 
    data['label'] = y 
    
    # create only index and label tabular dataset 
    processed_data = data[['Unnamed: 0', 'label']].copy()
    processed_data = processed_data.rename(columns={"Unnamed: 0": "index"})
    
    # drop for image dataset
    length = len(processed_data)
    processed_data.drop(processed_data.index[int(length/21):], inplace=True)
    return processed_data  

processed_dataset = []
for i in range(len(dataset)) :
    processed_data = preprocessed_data(dataset[i], i)
    processed_dataset.append(processed_data)
    
processed_dataset[1]

,index,label
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
295,295,1
296,296,1
297,297,1
298,298,1


In [ ]:
len(processed_dataset)

8

In [ ]:
# loading images
img_dataset = []
n = 0
for data in processed_dataset:
    n += 1
    for img_name in tqdm(data['index']):
        # defining the image path
        image_path = path+'Dataset0'+ str(n) +'/id0' + str(n) + '_frame_' + str(img_name+1) + '.png'
        # reading the image
        img = cv2.imread(image_path)
        # converting the type of pixel to float 32
        img = img.astype('float32')
        # Converts into the corret colorspace (RGB)
        img_cvt = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        # Reduce image size so training can be faster 
        img = cv2.resize(img_cvt, (300,300))
        # normalizing the pixel values
        img /= 255.0
        # appending the image into the list
        img_dataset.append(img)

# converting the list to numpy array
X = np.array(img_dataset)
X = X.reshape(len(X), 300, 300, 1)
# defining the target
y = processed_dataset[0]['label'].values
for i in range(1, len(processed_dataset)): 
    y = np.append(y, processed_dataset[i]['label'].values)
print(X.shape)
print(y.shape)

100%|██████████| 300/300 [04:36<00:00,  1.08it/s]


(2400, 300, 300, 1)
(2400,)


In [ ]:
# test_size: option to split between train and test dataset 
# random_state: seed
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                    test_size=0.3,
                                                    stratify=y,  
                                                    random_state=42)
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, 
                                                      test_size=0.2,
                                                      stratify=train_y, 
                                                      random_state=42)

In [ ]:
# This function is used more for debugging and showing results later. It plots the image into the notebook
def plot_image(data, n):
  print(data[n].shape)  # Prints the shape of the image just to check
  plt.grid(False)       # Without grid so we can see better
  plt.imshow(data[n])   # Shows the image
  plt.xlabel("Width")
  plt.ylabel("Height")
  plt.title("Image " + path)

In [ ]:
# plot the first image from our image array
plot_image(X, 1201) 

## Building Model: Simple CNN 

In [ ]:
# Construction of model
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(300, 300, 1))) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
# Configures the model for training
model.compile(optimizer='adam', # Optimization routine, which tells the computer how to adjust the parameter values to minimize the loss function.
              loss='sparse_categorical_crossentropy', # Loss function, which tells us how bad our predictions are.
              metrics=['accuracy']) # List of metrics to be evaluated by the model during training and testing.

## Training

In [ ]:
# Trains the model for a given number of epochs (iterations on a dataset) and validates it.
model.fit(train_X, train_y, epochs=5, batch_size=64, verbose=2, validation_data=(valid_X, valid_y))

Epoch 1/5
21/21 - 15s - loss: 2.3530 - accuracy: 0.2396 - val_loss: 1.6418 - val_accuracy: 0.5060 - 15s/epoch - 734ms/step
Epoch 2/5
21/21 - 4s - loss: 0.8203 - accuracy: 0.7336 - val_loss: 0.3261 - val_accuracy: 0.8869 - 4s/epoch - 170ms/step
Epoch 3/5
21/21 - 4s - loss: 0.1473 - accuracy: 0.9442 - val_loss: 0.0960 - val_accuracy: 0.9702 - 4s/epoch - 170ms/step
Epoch 4/5
21/21 - 4s - loss: 0.0307 - accuracy: 0.9896 - val_loss: 0.0617 - val_accuracy: 0.9821 - 4s/epoch - 171ms/step
Epoch 5/5
21/21 - 4s - loss: 0.0148 - accuracy: 0.9963 - val_loss: 0.0566 - val_accuracy: 0.9821 - 4s/epoch - 171ms/step


## Evaluation

In [ ]:
test_loss, test_acc = model.evaluate(test_X, test_y)

print('Test accuracy: {:2.2f}%'.format(test_acc*100))

23/23 [==============================] - 1s 32ms/step - loss: 0.0818 - accuracy: 0.9722
Test accuracy: 97.22%


In [ ]:
predictions = model.predict(test_X) # Make predictions towards the test set

23/23 [==============================] - 1s 29ms/step


In [ ]:
np.argmax(predictions[50]), test_y[50] # If same, got it right

(5, 5)

## Exporting

In [ ]:
# Save entire model to a HDF5 file
model.save(path+'1013data_cnn_1013.h5')